In [1]:
import cv2
from os import listdir
from os.path import isfile, join
from tkinter import *
from tkinter import messagebox
#from cv2 import *
import face_recognition
import os
from datetime import datetime
#import time
#import pickle
import pandas as pd
import numpy as np

In [ ]:
window=Tk()
window.title('Face Detection and Recognition Attendance System')
window.geometry('850x700')
window.config(background="#add8e6")
l1 = Label(window,font=("Algerian",22))
l1.grid(column=0, row = 1)
l2 = Label(window,text=" Face Detection And Recognition Smart Attendance System ",font=("Algerian",22, 'bold'),fg = 'white', bg='blue' )
l2.grid(column=4, row = 2)
l3= Label(window,font=("Algerian",22, 'bold'))
l3.grid(column=0, row = 3)
l4 = Label(window,font=("Algerian",22))
l4.grid(column=0, row = 4)
l4_2 = Label(window,font=("Algerian",22))
l4_2.grid(column=0, row = 5)

button1=Button(window, text='Save Picture', command = takePicture,font=("Helvetica",22,'bold'), fg = 'black', bg='#ffbf50')
button1.grid(column=4, row=6)
l5 = Label(window,font=("Algerian",22))
l5.grid(column=0, row = 7)

button2=Button(window, text='Train Dataset', command = encodes, font=("Helvetica",22,'bold'), fg = 'black', bg='#ffbf50')
button2.grid(column=4, row=8)
l6 = Label(window,font=("Algerian",22))
l6.grid(column=0, row = 9)

button3=Button(window, text='Mark Attendance', command = webcam, font=("Helvetica",22,'bold'), fg = 'black', bg='#ffbf50')
button3.grid(column=4, row=12)
l7 = Label(window,font=("Algerian",22))
l7.grid(column=0, row = 13)

button4=Button(window, text='Show Attendance', command = showAttendance,font=("Helvetica",22,'bold'), fg = 'black', bg='#ffbf50')
button4.grid(column=4, row=15)


window.mainloop()

Empty DataFrame
Columns: [NAME,          DATE,        TIME]
Index: []
              NAME          DATE        TIME
0     HADI HUSSAIN    29/07/2022    11:32:42
1        JOHN ELIA    29/07/2022    11:32:48
2     SELENA GOMEZ    29/07/2022    11:33:40
3       BILL GATES    29/07/2022    11:33:48
4  MARK ZUKKERBURG    29/07/2022    11:33:57


In [3]:
def takePicture():
    name = input('Enter Name: ')
    rollno = input('Enter Roll Number: ')
    cap = cv2.VideoCapture(0)
    count = 0
    while True:
        ret,frame = cap.read()
        count += 1
        face = cv2.resize(frame,(200,200))
        #face = cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
        cv2.imshow('frame', face)
        
        if cv2.waitKey(10)==32: #extra
            file_name_path = f'photo stock/{name} {rollno}'+ '.jpg'
            cv2.imwrite(file_name_path,face)
             
            cv2.putText(frame,str(count),(100,120),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),3)
            cv2.imshow('face Cropper', frame)
         
            #if cv2.waitKey(10)==13 or count==1:
            break

    cap.release()
    cv2.destroyAllWindows()
    print('Picture Taken Successfully...')

In [ ]:
takePicture()

In [4]:
path = 'C:\\Users\\Hadi Toori\\Desktop\\Final Year Project\\photo stock'
images = []
classNames = []
mylist = os.listdir(path)
for cl in mylist:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])
    
images = np.array(images)
#images.shape

def markAttendance(name):
    with open('Attendance.csv','r+') as f:
        myDataList = f.readlines()
        nameList = []
        rollnoList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            now = datetime.now()
            date = now.strftime('%d/%m/%Y')
            time = now.strftime('%H:%M:%S')
            f.writelines(f'\n{name}, {date}, {time}')
def showAttendance():
    attendance = pd.read_csv(r'C:\Users\Hadi Toori\Desktop\Final Year Project\Attendance.csv')
    print(attendance)
    return pd.DataFrame(attendance)

def encodes():
    print('Encoding in process...')
    findEncodings(images)
    print('Encoding completed successfully!')
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encoded_face = face_recognition.face_encodings(img)[0]
        encodeList.append(encoded_face)
    return encodeList
encoded_face_train = findEncodings(images)
print('Done...')

C:\Users\Hadi Toori\AppData\Local\Temp\ipykernel_14240\1299173649.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  images = np.array(images)


Done...


In [5]:
# take pictures from webcam 
def webcam():
    cap  = cv2.VideoCapture(0)
    while True:
        success, img = cap.read()
        imgS = cv2.resize(img, (0,0), None, 0.25,0.25)
        imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
        faces_in_frame = face_recognition.face_locations(imgS)
        encoded_faces = face_recognition.face_encodings(imgS, faces_in_frame)
        for encode_face, faceloc in zip(encoded_faces,faces_in_frame):
            matches = face_recognition.compare_faces(encoded_face_train, encode_face)
            faceDist = face_recognition.face_distance(encoded_face_train, encode_face)
            
            matchIndex = np.argmin(faceDist)
            #print(matchIndex)
            if matches[matchIndex]:
                name = classNames[matchIndex].upper() #.lower()
                y1,x2,y2,x1 = faceloc
                # since we scaled down by 4 times in line 6
                y1, x2,y2,x1 = y1*4,x2*4,y2*4,x1*4
                cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
                cv2.rectangle(img, (x1,y2-35),(x2,y2), (0,255,0), cv2.FILLED)
                cv2.putText(img,name, (x1+6,y2-5), cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
                cv2.putText(img, f'{round(faceDist[0],2)}', (50,50), cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
                #print(f'{matches}{round(faceDist[0],2)}')
                markAttendance(name)
            
        
        cv2.imshow('webcam', img)
        if cv2.waitKey(10)==13:
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
webcam()

In [ ]:
Attendance = pd.read_csv(r'C:\Users\Hadi Toori\Desktop\Final Year Project\Attendance.csv')
print(pd.DataFrame(Attendance))
    